In [ ]:
pip install numpy pandas matplotlib seaborn scipy openpyxl

In [13]:
# Импорт библиотек
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import warnings
warnings.filterwarnings('ignore')

# Настройка отображения
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)

In [14]:
df = pd.read_csv("turkiye.csv")

df.head(8)

,instr,class,nb.repeat,attendance,difficulty,Q1,Q2,Q3,Q4,Q5,Q6,Q7,Q8,Q9,Q10,Q11,Q12,Q13,Q14,Q15,Q16,Q17,Q18,Q19,Q20,Q21,Q22,Q23,Q24,Q25,Q26,Q27,Q28
0,1,2,1,0,4,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3
1,1,2,1,1,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3
2,1,2,1,2,4,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5
3,1,2,1,1,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3
4,1,2,1,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
5,1,2,1,3,3,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4
6,1,2,1,1,3,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4
7,1,2,1,1,3,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5


## Удаляем выбросы

In [15]:
df = pd.read_csv("turkiye.csv")

# Вопросы Q1–Q28
q_cols = [f'Q{i}' for i in range(1, 29)]

# Функция: ≥26 одинаковых ответов
def has_too_many_identical_answers(row):
    return row.value_counts().max() >= 26

# Маска аномальных ответов
mask_identical = df[q_cols].apply(has_too_many_identical_answers, axis=1)

# Подсчёты
n_identical = mask_identical.sum()
n_low_attendance = (df['attendance'] < 2).sum()

# Распределение по Attendance
dist_attendance_all = df['attendance'].value_counts().sort_index()

# Распределение по Attendance среди "одинаковых" записей
dist_attendance = df[mask_identical]['attendance'].value_counts().sort_index()

# Вывод
print(f"Студентов с ≥26 одинаковыми ответами: {n_identical}")
print(f"Студентов с посещаемостью < 2: {n_low_attendance}")

print("\nРаспределение по Attendance")
print(dist_attendance_all.to_string())

print("\nРаспределение по Attendance среди записей с ≥26 одинаковыми ответами:")
print(dist_attendance.to_string())

mask_A = df[q_cols].apply(has_too_many_identical_answers, axis=1)
mask_B = df['attendance'] < 2

# Пересечение
mask_intersection = mask_A | mask_B
n_intersection = mask_intersection.sum()

print(f"Число записей с ≥27 одинаковыми ответами ИЛИ посещаемостью < 2: {n_intersection}")

Студентов с ≥26 одинаковыми ответами: 3409
Студентов с посещаемостью < 2: 2930

Распределение по Attendance
attendance
0    1902
1    1028
2     792
3    1252
4     846

Распределение по Attendance среди записей с ≥26 одинаковыми ответами:
attendance
0    1292
1     599
2     434
3     624
4     460
Число записей с ≥27 одинаковыми ответами ИЛИ посещаемостью < 2: 4448


### Повторы и одинаковые ответы

In [16]:
df = pd.read_csv("turkiye.csv")

print(f"Исходное количество строк: {len(df)}")

# Колонки с вопросами Q1–Q28
q_cols = [f'Q{i}' for i in range(1, 29)]

# 1. Удаляем строки, где 27 или более ответов одинаковые
def has_too_many_identical_answers(row):
    # Считаем частоту каждого значения в строке
    value_counts = row.value_counts()
    # Если максимальная частота >= 26 — считаем аномалией
    # print(value_counts)
    # print(value_counts.max() >= 26)
    # print('-'*10)
    return value_counts.max() >= 26

mask_identical = df[q_cols].apply(has_too_many_identical_answers, axis=1)
# df[mask_identical].to_excel('удалённые_одинаковые_ответы2.xlsx', index=False)

removed_identical = mask_identical.sum()
df = df[~mask_identical].copy()

print(f"Удалено строк с ≥26 одинаковыми ответами: {removed_identical}")


# 2. Удаляем записи с посещаемостью ниже 2 (Attendance < 2)
# Согласно описанию, Attendance ∈ {0,1,2,3,4}
removed_low_attendance = (df['attendance'] < 2).sum()
df = df[df['attendance'] >= 2].copy()

print(f"Удалено строк с посещаемостью ниже 2: {removed_low_attendance}")
print(f"Итоговое количество строк после очистки: {len(df)}")

Исходное количество строк: 5820
Удалено строк с ≥26 одинаковыми ответами: 3409
Удалено строк с посещаемостью ниже 2: 1039
Итоговое количество строк после очистки: 1372


### Поиск выбросов по квартилям (метод IQR) — по каждому признаку отдельно

In [17]:
k = 1.5
outlier_mask_iqr = pd.Series([False] * len(df), index=df.index)

for col in q_cols:
    Q1 = df[col].quantile(0.25)
    Q3 = df[col].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - k * IQR
    upper_bound = Q3 + k * IQR
    outlier_mask_iqr |= (df[col] < lower_bound) | (df[col] > upper_bound)

n_outliers_iqr = outlier_mask_iqr.sum()

df = df[~outlier_mask_iqr].copy()

print(f"Число выбросов, обнаруженных методом IQR: {n_outliers_iqr}")
print(f"Итоговое количество строк после очистки: {len(df)}")

Число выбросов, обнаруженных методом IQR: 349
Итоговое количество строк после очистки: 1023


## Корреляции

### Общая матрица корреляции

In [ ]:
# ============================================================
# ПОЛНАЯ МАТРИЦА КОРРЕЛЯЦИЙ ДЛЯ ВСЕХ ЧИСЛОВЫХ ПРИЗНАКОВ
# ============================================================

# Выбираем все числовые колонки (включая дополнительные признаки)
q_cols = [f'Q{i}' for i in range(1, 29)]
numeric_cols = ['instr', 'class', 'nb.repeat', 'attendance', 'difficulty'] + q_cols

# Проверяем наличие всех колонок
available_cols = [col for col in numeric_cols if col in df.columns]
print(f"Числовые признаки для корреляции ({len(available_cols)} колонок):")
print(available_cols)
print()

# Вычисление полной матрицы корреляций
correlation_matrix_full = df[available_cols].corr()

# Вывод полной матрицы
print("=" * 120)
print("ПОЛНАЯ МАТРИЦА КОРРЕЛЯЦИЙ ДЛЯ ВСЕХ ПРИЗНАКОВ")
print("=" * 120)
print(correlation_matrix_full.to_string())

# Сохранение в CSV
correlation_matrix_full.to_csv('correlation_matrix_all_features.csv')
print("\n✓ Матрица корреляций сохранена в 'correlation_matrix_all_features.csv'")

# Описательная статистика
mask = np.triu(np.ones_like(correlation_matrix_full, dtype=bool), k=1)
upper_triangle = correlation_matrix_full.where(mask)
correlations = upper_triangle.stack()

print("\n" + "=" * 120)
print("СТАТИСТИКА КОРРЕЛЯЦИЙ")
print("=" * 120)
print(f"Количество пар признаков: {len(correlations)}")
print(f"Средняя корреляция: {correlations.mean():.4f}")
print(f"Медианная корреляция: {correlations.median():.4f}")
print(f"Минимальная корреляция: {correlations.min():.4f}")
print(f"Максимальная корреляция: {correlations.max():.4f}")
print(f"Стандартное отклонение: {correlations.std():.4f}")

# ============================================================
# ВИЗУАЛИЗАЦИЯ 1: Полная матрица
# ============================================================

plt.figure(figsize=(24, 22))
sns.heatmap(correlation_matrix_full, 
            annot=True,
            fmt='.2f',
            cmap='coolwarm',
            center=0,
            vmin=-1, vmax=1,
            square=True,
            linewidths=0.5,
            annot_kws={'size': 7},  # Размер шрифта для чисел
            cbar_kws={"shrink": 0.8})

plt.title('Полная матрица корреляций для всех признаков датасета', 
          fontsize=18, pad=20, weight='bold')
plt.xlabel('Признаки', fontsize=14)
plt.ylabel('Признаки', fontsize=14)
plt.xticks(rotation=90, fontsize=9)
plt.yticks(rotation=0, fontsize=9)
plt.tight_layout()
plt.savefig('correlation_matrix_all_features.png', dpi=300, bbox_inches='tight')
print("\n✓ Визуализация сохранена в 'correlation_matrix_all_features.png'")

# ============================================================
# ВИЗУАЛИЗАЦИЯ 2: Корреляции дополнительных признаков с вопросами
# ============================================================

# Матрица корреляций
additional_features = ['instr', 'class', 'nb.repeat', 'attendance', 'difficulty']
available_additional = [col for col in additional_features if col in df.columns]

# Создаём подматрицу
corr_additional_vs_questions = df[available_additional + q_cols].corr()
corr_subset = corr_additional_vs_questions.loc[available_additional, q_cols]

plt.figure(figsize=(18, 6))
sns.heatmap(corr_subset, 
            annot=True,
            fmt='.2f',
            cmap='coolwarm',
            center=0,
            vmin=-1, vmax=1,
            linewidths=0.5,
            cbar_kws={"shrink": 0.8})

plt.title('Корреляция дополнительных признаков с вопросами оценки (Q1-Q28)', 
          fontsize=16, pad=15, weight='bold')
plt.xlabel('Вопросы', fontsize=12)
plt.ylabel('Дополнительные признаки', fontsize=12)
plt.xticks(rotation=90)
plt.tight_layout()
plt.savefig('correlation_additional_features_vs_questions.png', dpi=300, bbox_inches='tight')
print("✓ Визуализация сохранена в 'correlation_additional_features_vs_questions.png'")

# ============================================================
# АНАЛИЗ КОРРЕЛЯЦИЙ ДОПОЛНИТЕЛЬНЫХ ПРИЗНАКОВ
# ============================================================

print("\n" + "=" * 120)
print("КОРРЕЛЯЦИЯ ДОПОЛНИТЕЛЬНЫХ ПРИЗНАКОВ С ВОПРОСАМИ")
print("=" * 120)

for feature in available_additional:
    print(f"\n{feature.upper()}:")
    corr_values = correlation_matrix_full.loc[feature, q_cols].sort_values(ascending=False)
    print(f"  Топ-5 наиболее сильных корреляций:")
    for q, val in corr_values.head(5).items():
        print(f"    {q}: {val:.4f}")
    print(f"  Топ-5 наиболее слабых корреляций:")
    for q, val in corr_values.tail(5).items():
        print(f"    {q}: {val:.4f}")

# ============================================================
# ТОП КОРРЕЛЯЦИЙ МЕЖДУ ВСЕМИ ПРИЗНАКАМИ
# ============================================================

print("\n" + "=" * 120)
print("ТОП-15 САМЫХ СИЛЬНЫХ ПОЛОЖИТЕЛЬНЫХ КОРРЕЛЯЦИЙ (ВСЕ ПРИЗНАКИ)")
print("=" * 120)

corr_pairs_all = []
for i in range(len(correlation_matrix_full.columns)):
    for j in range(i+1, len(correlation_matrix_full.columns)):
        corr_pairs_all.append({
            'Признак 1': correlation_matrix_full.columns[i],
            'Признак 2': correlation_matrix_full.columns[j],
            'Корреляция': correlation_matrix_full.iloc[i, j]
        })

corr_df_all = pd.DataFrame(corr_pairs_all)
corr_df_all_sorted = corr_df_all.sort_values('Корреляция', ascending=False)

print(corr_df_all_sorted.head(15).to_string(index=False))
corr_df_all_sorted.to_csv('correlation_pairs_all_features.csv', index=False)

print("\n" + "=" * 120)
print("ТОП-15 САМЫХ СЛАБЫХ КОРРЕЛЯЦИЙ (ВСЕ ПРИЗНАКИ)")
print("=" * 120)
print(corr_df_all_sorted.tail(15).to_string(index=False))


print("\n" + "=" * 120)
print("АНАЛИЗ ЗАВЕРШЁН")
print("=" * 120)
print("\nСозданные файлы:")
print("1. correlation_matrix_all_features.csv - полная матрица корреляций")
print("2. correlation_matrix_all_features.png - визуализация полной матрицы")
print("3. correlation_additional_features_vs_questions.png - доп. признаки vs вопросы")

### Матрица корреляции для предметов

In [18]:
class_counts = df['class'].value_counts().sort_index()
class_df = pd.DataFrame({
    'Предмет': class_counts.index,
    'Количество': class_counts.values
})

# Транспонирование для вывода в строку
print("Распределение по предметам:")
print(class_df.T.to_string(header=False))

Распределение по предметам:
Предмет      1   2    3   4    5   6   7    8   9  10  11  12   13
Количество  62  14  112  35  135  99  47  147  98  67  90   7  110


In [19]:
# Все числовые признаки
q_cols = [f'Q{i}' for i in range(1, 29)]
numeric_cols = ['nb.repeat', 'attendance', 'difficulty'] + q_cols # уберием class и instr (корреляция с константой)

print(numeric_cols)


# Предметы для анализа
subjects = [3, 5, 8]

for subject in subjects:
    # Фильтрация данных по предмету
    df_subject = df[df['class'] == subject]
    
    # Вычисление матрицы корреляций
    corr_matrix = df_subject[numeric_cols].corr()
    
    # Текстовый вывод
    print(f"\n{'='*120}")
    print(f"МАТРИЦА КОРРЕЛЯЦИЙ ДЛЯ ПРЕДМЕТА {subject} (n={len(df_subject)})")
    print('='*120)
    print(corr_matrix.to_string())
    
    # Сохранение в CSV
    corr_matrix.to_csv(f'correlation_class_{subject}.csv')
    
    # Визуализация
    plt.figure(figsize=(24, 22))
    sns.heatmap(corr_matrix, 
                annot=True,
                fmt='.2f',
                cmap='coolwarm',
                center=0,
                vmin=-1, vmax=1,
                square=True,
                linewidths=0.5,
                annot_kws={'size': 7},
                cbar_kws={"shrink": 0.8})
    
    plt.title(f'Матрица корреляций для предмета {subject} (n={len(df_subject)})', 
              fontsize=18, pad=20, weight='bold')
    plt.xlabel('Признаки', fontsize=14)
    plt.ylabel('Признаки', fontsize=14)
    plt.xticks(rotation=90, fontsize=9)
    plt.yticks(rotation=0, fontsize=9)
    plt.tight_layout()
    plt.savefig(f'correlation_class_{subject}.png', dpi=300, bbox_inches='tight')
    plt.close()
    
    print(f"✓ Сохранено: correlation_class_{subject}.csv и correlation_class_{subject}.png")


['nb.repeat', 'attendance', 'difficulty', 'Q1', 'Q2', 'Q3', 'Q4', 'Q5', 'Q6', 'Q7', 'Q8', 'Q9', 'Q10', 'Q11', 'Q12', 'Q13', 'Q14', 'Q15', 'Q16', 'Q17', 'Q18', 'Q19', 'Q20', 'Q21', 'Q22', 'Q23', 'Q24', 'Q25', 'Q26', 'Q27', 'Q28']

МАТРИЦА КОРРЕЛЯЦИЙ ДЛЯ ПРЕДМЕТА 3 (n=112)
            nb.repeat  attendance  difficulty        Q1        Q2        Q3        Q4        Q5        Q6        Q7        Q8        Q9       Q10       Q11       Q12       Q13       Q14       Q15       Q16       Q17       Q18       Q19       Q20       Q21       Q22       Q23       Q24       Q25       Q26       Q27       Q28
nb.repeat    1.000000   -0.056027    0.086559  0.108842 -0.071600 -0.056642  0.107333 -0.001169  0.065805 -0.027487  0.074656  0.035098  0.034777  0.049232  0.118198 -0.097082 -0.053247 -0.001297 -0.088175 -0.170159 -0.109463 -0.122571 -0.037569 -0.029491 -0.016230 -0.079884 -0.080470 -0.065697 -0.002174 -0.026205 -0.172892
attendance  -0.056027    1.000000   -0.149757  0.114040  0.176235  0.045849 

## Описательные статистики